In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('Using device: {0}'.format(device))

Using device: mps


In [3]:
model_name = "allenai/OLMo-2-0425-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto")
model.eval()

Olmo2ForCausalLM(
  (model): Olmo2Model(
    (embed_tokens): Embedding(100352, 2048, padding_idx=100277)
    (layers): ModuleList(
      (0-15): 16 x Olmo2DecoderLayer(
        (self_attn): Olmo2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Olmo2RMSNorm((2048,), eps=1e-06)
          (k_norm): Olmo2RMSNorm((2048,), eps=1e-06)
        )
        (mlp): Olmo2MLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (post_attention_layernorm): Olmo2RMSNorm((2048,), eps=1e-06)
        

In [4]:
input_text = "Write a short poem about Eleazar Wheelock, founder of Dartmouth College."
max_new_tokens = 128
msg = [{"role":"user","content":input_text}]
input_ids = tokenizer.apply_chat_template(msg, 
                                          return_tensors = "pt",
                                          add_generation_prompt = False)

output = model.generate(input_ids.to(device), 
                        do_sample=True, 
                        max_new_tokens = max_new_tokens,
                        temperature = 1.0, 
                        top_p = 0.95)

print(tokenizer.decode(output[0], 
                       skip_special_tokens=False))

<|endoftext|><|user|>
Write a short poem about Eleazar Wheelock, founder of Dartmouth College.
Dartmouth College by Eleazer Wheelock was established
In 1749. Eleazer Wheelock was a man of much faith
And his vision helped shape Dartmouth College's growth.

No wonder the founders of Dartmouth College
Are known as some of the most trusted men.
It is clear the people of New Hampshire admired
Eleazer Wheelock's vision for Dartmouth College.

This small college was founded with hope,
To nurture young scholars.
It grew from a modest start,
Became Dartmouth College's home.

Today, Eleazer Wheelock is honored,
His name appears on the University of New Hampshire's seal
